In [ ]:
import cv2
import torch
import os
from ultralytics import YOLO
import time
import numpy as np

model = YOLO("yolov8x.pt")

# COCO class IDs for vehicles
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Function to compute IoU
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Function to filter overlapping boxes
def filter_overlapping_boxes(boxes, iou_threshold=0.5):
    filtered_boxes = []
    for box in boxes:
        keep = True
        for filtered_box in filtered_boxes:
            if compute_iou(box.xyxy[0].tolist(), filtered_box.xyxy[0].tolist()) > iou_threshold:
                keep = False
                break
        if keep:
            filtered_boxes.append(box)
    return filtered_boxes

# Function to detect vehicles in an image and return count
def detect_vehicles(image_path):
    img = cv2.imread(image_path)
    results = model(img, conf=0.4, iou=0.5)[0]
    filtered_boxes = filter_overlapping_boxes(results.boxes)
    vehicle_count = sum(1 for box in filtered_boxes if int(box.cls[0]) in vehicle_classes)
    return vehicle_count

# Function to create a traffic signal
def create_traffic_signal(direction, timer, active_light):
    signal_width, signal_height = 100, 300  # Width, Height of the traffic signal
    signal = np.zeros((signal_height, signal_width, 3), dtype=np.uint8)
    signal[:] = (50, 50, 50)  # Gray background

    # Draw the traffic light circles
    light_radius = 30
    light_positions = [
        (signal_width // 2, 60),   # Red light position
        (signal_width // 2, 150),  # Orange light position
        (signal_width // 2, 240),  # Green light position
    ]

    # Draw the lights
    for i, (x, y) in enumerate(light_positions):
        color = (0, 0, 0)  # Default to black (off)
        if i == 0 and active_light == "red":
            color = (0, 0, 255)  # Red
        elif i == 1 and active_light == "orange":
            color = (0, 165, 255)  # Orange
        elif i == 2 and active_light == "green":
            color = (0, 255, 0)  # Green
        cv2.circle(signal, (x, y), light_radius, color, -1)  # Fill the circle
        cv2.circle(signal, (x, y), light_radius, (255, 255, 255), 2)  # White border

    # Add direction text
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f"Dir {direction}"
    text_size = cv2.getTextSize(text, font, 0.8, 2)[0]
    text_x = (signal_width - text_size[0]) // 2
    text_y = signal_height - 20
    cv2.putText(signal, text, (text_x, text_y), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # Add timer text
    timer_text = f"Time: {timer}"
    timer_size = cv2.getTextSize(timer_text, font, 0.6, 2)[0]
    timer_x = (signal_width - timer_size[0]) // 2
    timer_y = text_y - 30
    cv2.putText(signal, timer_text, (timer_x, timer_y), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

    return signal

# Paths to image folders for each direction
image_folders = {
    'c1': "../Data/f1",
    'c2': "../Data/f2",
    'c3': "../Data/f3",
    'c4': "../Data/f4"
}

# Initial Timer Configurations
timer = 0
t1, t2, t3, t4 = 30, 30, 30, 30
c1, c2, c3, c4 = 0, 0, 0, 0

# Create a window to display traffic signals
cv2.namedWindow("Traffic Signals", cv2.WINDOW_NORMAL)

while True:
    # Cycle through each direction
    for direction, t in zip(range(1, 5), [t1, t2, t3, t4]):
        for i in range(1, t + 1):
            print(f"Timer: {i} | Direction: {direction}")  # Reset timer count for each cycle

            # Detect vehicles at intervals
            if direction == 1 and i == 1:
                c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
                c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
                c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
            elif direction == 2 and i == 1:
                c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
                c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
                c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
            elif direction == 3 and i == 1:
                c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
                c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
                c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
            elif direction == 4 and i == 1:
                c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
                c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
                c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))

            # Create traffic signals for all directions
            active_light = "green" if direction == 1 else "red"
            signal1 = create_traffic_signal(1, t1 - i + 1, "green" if direction == 1 else "red")
            signal2 = create_traffic_signal(2, t2 - i + 1, "green" if direction == 2 else "red")
            signal3 = create_traffic_signal(3, t3 - i + 1, "green" if direction == 3 else "red")
            signal4 = create_traffic_signal(4, t4 - i + 1, "green" if direction == 4 else "red")

            # Combine all signals into one image
            top_row = np.hstack((signal1, signal2))
            bottom_row = np.hstack((signal3, signal4))
            combined_signals = np.vstack((top_row, bottom_row))

            # Display the combined signals
            cv2.imshow("Traffic Signals", combined_signals)
            cv2.waitKey(1)  # Refresh the display

            # time.sleep(1)
            timer += 1

    # Calculate new time allocations based on detected vehicle counts
    min_time = 10  # Minimum green light duration
    total_vehicles = max(c1 + c2 + c3 + c4, 1)  # Avoid division by zero

    t1 = max(min_time, (c1 * 120) // total_vehicles)
    t2 = max(min_time, (c2 * 120) // total_vehicles)
    t3 = max(min_time, (c3 * 120) // total_vehicles)
    t4 = max(min_time, (c4 * 120) // total_vehicles)

    # Ensure the total time does not exceed 120 seconds
    total_time = t1 + t2 + t3 + t4
    if total_time > 120:
        scale_factor = 120 / total_time
        t1 = max(min_time, int(t1 * scale_factor))
        t2 = max(min_time, int(t2 * scale_factor))
        t3 = max(min_time, int(t3 * scale_factor))
        t4 = max(min_time, int(t4 * scale_factor))

    print(f"Updated Timers: {t1}, {t2}, {t3}, {t4}")
    print(f"Current count : {c1},{c2},{c3},{c4}")

Timer: 1 | Direction: 1

0: 352x640 5 cars, 1 truck, 1 traffic light, 734.0ms
Speed: 2.3ms preprocess, 734.0ms inference, 1.2ms postprocess per image at shape (1, 3, 352, 640)

0: 416x640 8 cars, 1 truck, 794.8ms
Speed: 2.2ms preprocess, 794.8ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 3 cars, 846.4ms
Speed: 2.3ms preprocess, 846.4ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)
Timer: 2 | Direction: 1
Timer: 3 | Direction: 1
Timer: 4 | Direction: 1
Timer: 5 | Direction: 1
Timer: 6 | Direction: 1
Timer: 7 | Direction: 1
Timer: 8 | Direction: 1
Timer: 9 | Direction: 1
Timer: 10 | Direction: 1
Timer: 11 | Direction: 1
Timer: 12 | Direction: 1
Timer: 13 | Direction: 1
Timer: 14 | Direction: 1
Timer: 15 | Direction: 1
Timer: 16 | Direction: 1
Timer: 17 | Direction: 1
Timer: 18 | Direction: 1
Timer: 19 | Direction: 1
Timer: 20 | Direction: 1
Timer: 21 | Direction: 1
Timer: 22 | Direction: 1
Timer: 23 | Direction: 1
Timer: 24 | Direction